In [63]:
import duckdb
import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff

In [64]:
conn = duckdb.connect("../flood_forecasting.duckdb", read_only=True)

In [65]:
conn.sql("SHOW ALL TABLES").show()

┌───────────────────┬──────────────┬──────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [66]:
query = "SELECT * FROM flood_forecasting.main_marts.dim_sites LIMIT 10"
conn.sql(query).show()

┌──────────┬──────────────────────────────────────────────────┬─────────────┬──────────────┬─────────────────────┬──────────┐
│ site_id  │                   station_name                   │  latitude   │  longitude   │ drainage_area_sq_mi │ huc_code │
│ varchar  │                     varchar                      │   double    │    double    │       double        │  int64   │
├──────────┼──────────────────────────────────────────────────┼─────────────┼──────────────┼─────────────────────┼──────────┤
│ 05013500 │ ST. Mary Lake near ST. Mary MT                   │ 48.73608815 │ -113.4314906 │               130.0 │ 10010002 │
│ 05013600 │ St. Mary River near St. Mary MT                  │  48.7260111 │ -113.4356806 │               130.0 │ 10010002 │
│ 05013640 │ Divide Creek near St. Mary, MT                   │ 48.69219745 │ -113.4226016 │                NULL │ 10010002 │
│ 05013650 │ Divide Creek at St. Mary, MT                     │ 48.74053277 │ -113.4292683 │                NULL │ 100

In [67]:
query = "SELECT * FROM flood_forecasting.main_marts.dim_catchment_attributes LIMIT 10"
conn.sql(query).show()

┌──────────┬───────────────────────────────────────────┬─────────────┬──────────────┬──────────┬─────────────────────┬───────────────────────┬─────────────┬────────────┬────────────┬───────────┬────────────────────┬────────────────────┬─────────────────────────┬───────────────────────┬──────────────┬──────────────┬──────────────┬────────────────┬────────────────────────┬───────────────────────────────────────────────────────────────────────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────────┬───────────────────┬──────────────────┬──────────────────┬───────────────────┬──────────────────────┬────────────────────────┬─────────────────────────┬──────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬──────────────────────┐
│ site_id  │               station_name                │  latitude   │  longitude   │ huc_code │ drainage_area_sq_km │ is_reference_hcdn2009 │ elev_mean_

In [68]:
query = """
    SELECT a.site_id
    FROM flood_forecasting.main_marts.dim_sites AS a
    RIGHT JOIN flood_forecasting.main_marts.dim_catchment_attributes AS b ON a.site_id = b.site_id
    WHERE a.site_id IS NULL
"""
conn.sql(query).show()

┌─────────┐
│ site_id │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘



In [69]:
query = """
    SELECT *
    FROM flood_forecasting.main_marts.dim_catchment_attributes AS a
    WHERE site_id = '05013500'
"""
conn.sql(query).show()

┌──────────┬────────────────────────────────┬─────────────┬──────────────┬──────────┬─────────────────────┬───────────────────────┬─────────────┬────────────┬────────────┬───────────┬──────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬──────────────┬──────────────┬──────────────┬────────────────┬────────────────────────┬───────────────────┬────────┬──────────┬───────────────┬───────────────┬───────────┬────────────────┬───────────────┬───────────────────┬──────────────────────┬────────────────────────┬─────────────────────────┬──────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬──────────────────────┐
│ site_id  │          station_name          │  latitude   │  longitude   │ huc_code │ drainage_area_sq_km │ is_reference_hcdn2009 │ elev_mean_m │ elev_max_m │ elev_min_m │ SLOPE_PCT │ ASPECT_NORTHNESS │ ASPECT_EASTNESS │ soil_water_capacity_avg │ soil_permeability_avg │ cla

In [70]:
query = """
    SELECT *
    FROM flood_forecasting.main_marts.dim_sites AS a
    WHERE site_id = '05013500'
"""
conn.sql(query).show()

┌──────────┬────────────────────────────────┬─────────────┬──────────────┬─────────────────────┬──────────┐
│ site_id  │          station_name          │  latitude   │  longitude   │ drainage_area_sq_mi │ huc_code │
│ varchar  │            varchar             │   double    │    double    │       double        │  int64   │
├──────────┼────────────────────────────────┼─────────────┼──────────────┼─────────────────────┼──────────┤
│ 05013500 │ ST. Mary Lake near ST. Mary MT │ 48.73608815 │ -113.4314906 │               130.0 │ 10010002 │
└──────────┴────────────────────────────────┴─────────────┴──────────────┴─────────────────────┴──────────┘



**Conclusion: maybe delete dim_sites after merging drainage area info into dim_catchment_attributes**

In [71]:
query = """
SELECT site_id, latitude, longitude
FROM main_marts.dim_sites
"""
df = conn.sql(query).df()

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    scope='usa',
    title='USGS Sites'
)
fig.show()

In [72]:
query = """
SELECT site_id, latitude, longitude, observation_hour, gage_height_ft_mean
FROM flood_forecasting.main.flood_model
WHERE gage_height_ft_mean > 0
ORDER BY observation_hour
"""
df = conn.sql(query).df()

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    scope='usa',
    title='USGS Sites'
)
fig.show()

***While there are more locations in other table, this is the one that has the info***

In [73]:
df['time'] = df['observation_hour'].astype(str)

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    color='gage_height_ft_mean',
    animation_frame='time',
    scope='usa',
    title='Gage Height Over Time',
    color_continuous_scale='RdYlGn_r',
    range_color=[df['gage_height_ft_mean'].min(), df['gage_height_ft_mean'].max()],
    hover_data=['site_id']
)

fig.show()

***Maybe normalization?***

In [74]:
fig = px.line(df, x='observation_hour', y='gage_height_ft_mean', color='site_id', title='Gage Height Comparison')
fig.show()

In [76]:
fig = px.box(df, x='site_id', y='gage_height_ft_mean', title='Gage Height Distribution by Site')
fig.show()

In [77]:
query = """
    SELECT gage_height_ft_mean, streamflow_cfs_mean, precipitation_mm, 
           temperature_c, humidity_pct, drainage_area_sq_km
    FROM main.flood_model 
    WHERE gage_height_ft_mean > 0
"""

corr_df = conn.sql(query).df()

fig = px.imshow(corr_df.corr(), text_auto=True, title='Correlation Matrix', 
                color_continuous_scale='RdYlGn', zmin=-1, zmax=1)       
fig.show()